In [1]:
import cv2
import numpy as np

In [7]:
import cv2
import numpy as np

# Open the webcam
cap = cv2.VideoCapture(0)

# Trackbar callback function (does nothing)
def nothing(x):
    pass

# Create a window and trackbars
cv2.namedWindow('hand detection', cv2.WINDOW_NORMAL)
cv2.resizeWindow('hand detection', 400, 300)

cv2.createTrackbar('thres', 'hand detection', 0, 255, nothing)
cv2.createTrackbar('l_hue', 'hand detection', 0, 255, nothing)
cv2.createTrackbar('l_satu', 'hand detection', 0, 255, nothing)
cv2.createTrackbar('l_value', 'hand detection', 0, 255, nothing)
cv2.createTrackbar('u_hue', 'hand detection', 255, 255, nothing)
cv2.createTrackbar('u_satu', 'hand detection', 255, 255, nothing)
cv2.createTrackbar('u_value', 'hand detection', 255, 255, nothing)

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if there's an issue with capturing frames

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get trackbar values
    l_h = cv2.getTrackbarPos('l_hue', 'hand detection')
    l_s = cv2.getTrackbarPos('l_satu', 'hand detection')
    l_v = cv2.getTrackbarPos('l_value', 'hand detection')
    u_h = cv2.getTrackbarPos('u_hue', 'hand detection')
    u_s = cv2.getTrackbarPos('u_satu', 'hand detection')
    u_v = cv2.getTrackbarPos('u_value', 'hand detection')

    # Define HSV range for hand detection
    lower_bound = np.array([l_h, l_s, l_v])
    upper_bound = np.array([u_h, u_s, u_v])

    # Create mask
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    filtr = cv2.bitwise_and(frame, frame, mask=mask)

    # Invert mask
    mask1 = cv2.bitwise_not(mask)

    # Apply thresholding
    thres = cv2.getTrackbarPos('thres', 'hand detection')
    _, threshold = cv2.threshold(mask1, thres, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        epsilon=0.01*cv2.arcLength(c,True)# return contour parameter
        data=cv2.approxPolyDP(c,epsilon,True)# GIVE approximation in polygon shape
        #convexhull  is used to provide proper contours convexity
        hull=cv2.convexHull(data)
        cv2.drawContours(frame,[c],-1,(50,150,255),1)
        cv2.drawContours(frame,[hull],-1,(5,25,250),1)
    # Show results
    cv2.imshow('Filtered Image', filtr)
    cv2.imshow('Mask', mask)
    cv2.imshow('Threshold', threshold)
    cv2.imshow('result', frame)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
